In [1]:
# Import necessary libraries
import csv
import os

import biosppy.signals.ecg as ecg
import neurokit2 as nk

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from tqdm import tqdm

In [2]:
# Load dataset
X = pd.read_csv('X_train.csv', index_col='id')
y = pd.read_csv('y_train.csv', index_col='id')
X_test_sub = pd.read_csv('X_test.csv', index_col='id')

In [3]:
print("Shape of training dataset: {}".format(X.shape))
print("Shape of testing dataset: {}".format(X_test_sub.shape))

Shape of training dataset: (5117, 17807)
Shape of testing dataset: (3411, 17807)


# Preprocessing

### Feature Extraction 

In [65]:
def feature_extraction(signal):
    extracted_features = []        
    # ECG Signal Cleaning
    signal_clean = nk.ecg_clean(signal, 300)
    # Find ECG Signal Peaks
    sig_r_peaks, r_peaks = nk.ecg_peaks(signal_clean, sampling_rate=300)
    if sig_r_peaks['ECG_R_Peaks'].sum() >= 4:
        sig_waves_peaks, waves_peak = nk.ecg_delineate(signal, r_peaks, sampling_rate=300, method='peak', show=False)
        r_peaks = [x for x in r_peaks['ECG_R_Peaks'] if np.isnan(x) == False]
        p_peaks = [x for x in waves_peak['ECG_P_Peaks'] if np.isnan(x) == False]
        q_peaks = [x for x in waves_peak['ECG_Q_Peaks'] if np.isnan(x) == False]
        s_peaks = [x for x in waves_peak['ECG_S_Peaks'] if np.isnan(x) == False]
        t_peaks = [x for x in waves_peak['ECG_T_Peaks'] if np.isnan(x) == False]
    
        # R,P,Q,S and T peaks (mean and std of amplitude)
        extracted_features.append(np.mean(signal_clean[r_peaks]))
        extracted_features.append(np.std(signal_clean[r_peaks]))
        extracted_features.append(np.mean(signal_clean[p_peaks]))
        extracted_features.append(np.std(signal_clean[p_peaks]))
        extracted_features.append(np.mean(signal_clean[q_peaks]))
        extracted_features.append(np.std(signal_clean[q_peaks]))
        extracted_features.append(np.mean(signal_clean[s_peaks]))
        extracted_features.append(np.std(signal_clean[s_peaks]))
        extracted_features.append(np.mean(signal_clean[t_peaks]))
        extracted_features.append(np.std(signal_clean[t_peaks]))
    
        # Time-domain measures HRV metrics
        hrv_time = nk.hrv_time(sig_r_peaks, sampling_rate=300, show=False)
        extracted_features.append(hrv_time['HRV_MeanNN'][0])
        extracted_features.append(hrv_time['HRV_SDNN'][0])
        extracted_features.append(hrv_time['HRV_RMSSD'][0])
        extracted_features.append(hrv_time['HRV_SDSD'][0])
        extracted_features.append(hrv_time['HRV_MinNN'][0])
        extracted_features.append(hrv_time['HRV_MaxNN'][0])
    else:
        for i in range(16):
            extracted_features.append(np.nan)
    
    # Frequency-domain HRV metrics
    #hrv_freq = nk.hrv_frequency(sig_r_peaks, sampling_rate=300, silent=False,show=True)

    return np.asarray(extracted_features)

In [43]:
# Testing of feature function on one sample from training set
one_sample = X.loc[0].dropna().to_numpy(dtype='float32')
features_one_sample = feature_extraction(one_sample)
print(one_sample.shape)
print(features_one_sample)

[ 26.60090522  29.28583889  32.02673737 ... -44.93023267 -36.71436922
 -28.16187504]
(16322,)
[360.73804942  60.60299383  23.23128597  54.2089349  -75.32994529
  37.06043352 -94.13679695  39.10238103 212.40333529  54.65280278
 815.90909091  64.94125012  63.18471091  63.22160354 326.66666667
 853.33333333]


In [63]:
# Testing of feature function on one sample from testing set
one_sample = X_test_sub.loc[700].dropna().to_numpy(dtype='float32')
features_one_sample = feature_extraction(one_sample)
print(one_sample.shape)
print(features_one_sample)

(5687,)
(5687,)


In [23]:
# Transform training dataset of ecg signal into features
X_feat = np.zeros(shape=(X.shape[0],16))
for i in tqdm(range(X.shape[0])):
    X_feat[i] = feature_extraction(X.loc[i].dropna().to_numpy(dtype='float32'))

  2%|▊                                       | 105/5117 [00:17<15:45,  5.30it/s]/Users/damolaagbelese/opt/anaconda3/envs/MachineLearning/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/damolaagbelese/opt/anaconda3/envs/MachineLearning/lib/python3.9/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/Users/damolaagbelese/opt/anaconda3/envs/MachineLearning/lib/python3.9/site-packages/numpy/core/_methods.py:262: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/Users/damolaagbelese/opt/anaconda3/envs/MachineLearning/lib/python3.9/site-packages/numpy/core/_methods.py:222: RuntimeWarning: invalid value encountered in true_divide
  arrmean = um.true_divide(arrmean, div, out=arrmean, casting='unsafe',
/Users/damolaagbelese/opt/anaconda3/envs

100%|███████████████████████████████████████| 5117/5117 [14:32<00:00,  5.86it/s]


In [66]:
# Transform testing dataset of ecg signal into features
X_test_feat = np.zeros(shape=(X_test_sub.shape[0],16))
for i in tqdm(range(X_test_sub.shape[0])):
    X_test_feat[i] = feature_extraction(X_test_sub.loc[i].dropna().to_numpy(dtype='float32'))

 12%|████▋                                   | 402/3411 [01:10<10:50,  4.62it/s]/Users/damolaagbelese/opt/anaconda3/envs/MachineLearning/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/damolaagbelese/opt/anaconda3/envs/MachineLearning/lib/python3.9/site-packages/numpy/core/_methods.py:262: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
 20%|███████▉                                | 673/3411 [01:57<06:31,  6.99it/s]/Users/damolaagbelese/opt/anaconda3/envs/MachineLearning/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/damolaagbelese/opt/anaconda3/envs/MachineLearning/lib/python3.9/site-packages/numpy/core/_methods.py:262: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=

In [67]:
# Create dataframe for training and testing
columns = ['R-mean', 'R-std', 'P-mean', 'P-std', 'Q-mean', 'Q-std', 'S-mean', 'S-std', 'T-mean', 'T-std', 'MeanNN', 'SDNN', 'RMSSD', 'SDSD', 'MinNN', 'MaxNN']
X_feat = pd.DataFrame(X_feat, columns=columns)
X_feat.index.name = 'id'
X_test_feat = pd.DataFrame(X_test_feat, columns=columns)
X_test_feat.index.name = 'id'

In [68]:
# Save train and test features 
X_feat.to_csv('X_feat.csv')
X_test_feat.to_csv('X_test_feat.csv')

In [69]:
# Load train and test features 
X_feat = pd.read_csv('X_feat.csv', index_col='id')
X_test_feat = pd.read_csv('X_test_feat.csv', index_col='id')

In [70]:
print("Shape of feature training set: {}".format(X_feat.shape))
print("Shape of feature testing set: {}".format(X_test_feat.shape))

Shape of feature training set: (5117, 16)
Shape of feature testing set: (3411, 16)


### Imputation of missing values

In [71]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy='median')
X_train_imp = imputer.fit_transform(X_feat)
X_test_imp = imputer.transform(X_test_feat)

X_train_imp = pd.DataFrame(X_train_imp, columns=columns)
X_test_imp = pd.DataFrame(X_test_imp, columns=columns)

### Standardization

In [72]:
# Standard Scaler
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_imp)
X_test_scaled = scaler.transform(X_test_imp)

X_train_scaled = pd.DataFrame(X_train_scaled, columns=columns)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=columns)

# Models

In [75]:
# Train-Test-Split
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_train_scaled, y, test_size=0.2) 

In [77]:
# Classifiers
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
classifiers = {}
bagg_class = {}
classifiers['RandomForest'] = RandomForestClassifier(criterion='log_loss', class_weight='balanced')
classifiers['DecisionTree'] = DecisionTreeClassifier(criterion='log_loss', class_weight='balanced')

for key, value in tqdm(classifiers.items()):
    bagg_class[key] = BaggingClassifier(base_estimator=value, n_estimators=10).fit(X_train, np.array(y_train).ravel())
    y_pred = bagg_class[key].predict(X_test)
    F1 = f1_score(y_test, y_pred, average='micro')
    print("F1 score of {}: {}".format(key, F1))

 50%|██████████████████████▌                      | 1/2 [00:09<00:09,  9.66s/it]

F1 score of RandomForest: 0.7734375


100%|█████████████████████████████████████████████| 2/2 [00:10<00:00,  5.11s/it]

F1 score of DecisionTree: 0.7451171875


# Submission

In [78]:
print('Shape of training set:', X_train_scaled.shape)
print('Shape of training label:', y.shape)

Shape of training set: (5117, 16)
Shape of training label: (5117, 1)


In [79]:
# Train on whole train set 
bagg_class['RandomForest'].fit(X_train_scaled, np.array(y).ravel())

BaggingClassifier(base_estimator=RandomForestClassifier(class_weight='balanced',
                                                        criterion='log_loss'))

In [80]:
# Prediction on submission test set
submission = bagg_class['RandomForest'].predict(X_test_scaled)
df_submission = pd.DataFrame({'id': X_test_scaled.index, 'y': submission})

In [81]:
# Save into csv file
df_submission.to_csv('submission.csv',index=False)